In [1]:
%pylab inline
import rlcompleter, readline
readline.parse_and_bind('tab: complete')
import numpy as np
from matplotlib import pylab as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import json
import os
os.getcwd()
os.chdir("../Desktop/ModernAnalytics/assignment2")
with open("train.json") as f:
    obj = np.asarray(json.load(f))
cuisine, ID, ingredients= [],[],[]
for each in obj:
    cuisine.append(each['cuisine'])
    ID.append(each['id'])
    ingredients.extend(each['ingredients'])
#    component.append(each['ingredients'])
cuisine = np.asarray(cuisine)
ID = np.asarray(ID,dtype='int')
#ingredients = np.asarray(ingredients)
unique_ingredients = np.unique(ingredients)
#print(ingredients.shape)
print(np.unique(cuisine).shape)
print(ID.shape)
print(unique_ingredients.shape)
#print(unique_ingredients)

(20,)
(39774,)
(6714,)


In [3]:
#train = np.zeros([obj.shape[0],unique_ingredients.shape[0]])
#print(np.sum(train))
#for i,each in enumerate(obj):
#    indexs = np.searchsorted(unique_ingredients,each['ingredients'],side = 'left')
#    for x in each['ingredients']:
#        if(x not in unique_ingredients):
#            print(x)
#    train[i][indexs] = 1
#print(np.sum(train))
#print(len(ingredients))
def readMatrix(a):
#    outcome = np.zeros([a.shape[0],unique_ingredients.shape[0]])
    outcome = []
    for i,each in enumerate(a):
        outcome.append(np.in1d(unique_ingredients,each['ingredients']))
    return np.asarray(outcome)
train = readMatrix(obj)
print(train.shape)

(39774, 6714)


In [4]:
print(np.sum(train))

428249


In [5]:
train = np.asarray(train,dtype='int')

In [6]:
#ingredients_test = []
with open("test.json") as f:
    obj_test = np.asarray(json.load(f))
#    for each in obj_test:
#        ingredients_test.extend(each['ingredients'])
#unique_ingredients_test = np.unique(ingredients_test)
#test = np.zeros([obj_test.shape[0],unique_ingredients_test.shape[0]])
#for i,each in enumerate(obj_test):
#    indexs = np.searchsorted(unique_ingredients_test,each['ingredients'])
#    test[i][indexs] = 1
test = np.asarray(readMatrix(obj_test),dtype='int')

In [7]:
print(test.shape)

(9944, 6714)


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
gnb = GaussianNB()
bnb = BernoulliNB()
def Validation_3fold(n_train,n_label,method):
    s,e = 0,len(n_train)//3
    accuracy = 0
    for i in range(0,3):
        ts = n_train[s:e]
        ts_label = n_label[s:e]
        tn = np.delete(n_train,np.arange(s,e),axis = 0)
        tn_label = np.delete(n_label,np.arange(s,e))
        method.fit(tn,tn_label)
        accuracy += method.score(ts,ts_label)
        s = e
        e +=len(n_train)//3
#        print(s)
#        print(e)
    return accuracy/3
Validation_3fold(train,cuisine,bnb)

0.68353698395937046

In [9]:
Validation_3fold(train,cuisine,gnb)

0.38238547795041988

In [10]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
Validation_3fold(train,cuisine,lr)

0.77555689646502746

In [11]:
lr.fit(train,cuisine)
result = lr.predict(test)


In [12]:
print(result.shape)

(9944,)


In [13]:
import csv
with open("logistic_output",'wb') as f:
    writer = csv.writer(f)
    writer.writerow(['id','cuisine'])
    for i in range(0,obj_test.shape[0]):
        writer.writerow([obj_test[i]['id'],result[i]])

In [14]:
gnb.fit(train,cuisine)
gnb_outcome = gnb.predict(test)
with open("gnb_output",'wb') as f:
    writer = csv.writer(f)
    writer.writerow(['id','cuisine'])
    for i in range(0,obj_test.shape[0]):
        writer.writerow([obj_test[i]['id'],gnb_outcome[i]])

In [15]:
bnb.fit(train,cuisine)
bnb_outcome = bnb.predict(test)
with open("bnb_output",'wb') as f:
    writer = csv.writer(f)
    writer.writerow(['id','cuisine'])
    for i in range(0,obj_test.shape[0]):
        writer.writerow([obj_test[i]['id'],bnb_outcome[i]])